# Denison CS181/DA210 Homework 4.b - Step 1

Before you turn this problem in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells** (in the menubar, select Kernel$\rightarrow$Restart And Run All).

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

#### Import Python modules and load "SQL Magic"

In [1]:
import pandas as pd
import os
import os.path
import json
import sys
import importlib

module_dir = "../../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%load_ext sql

In [2]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [3]:
scheme, dbdir, database = getsqlite_creds(source="sqlite1")
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

Connection string: sqlite:///../../dbfiles/book.db


#### Establish Connection from Client to Server

In [4]:
%sql $cstring

---

## Part A: SQL Column Operations

In the following cells, your only action is to remove the `# YOUR CODE HERE` and `raise NotImplementedError()` lines, and then put a valid SQL statement as the **value** of string variable `query`.

In each case, when you execute the cell, the query will be sent to the database management system, a result obtained, and the result converted into a `pandas` data frame, whose prefix is shown.  (This allows for testing the results as well as displaying them in your Jupyter Notebook.)

_Note: you may want to view the `book` database in SQLiteStudio to get an idea of what the field names are as you work through these exercises._

**Q1:** Using the table `indicators`, write a query to project the code, year, gdp, pop, and find the GDP per capita of each country, calling the new column `percapita`.  Sort the data in descending value of `percapita`.

In [9]:
query = """SELECT code, year, gdp / pop AS percapita FROM
(SELECT code, year, gdp, pop FROM indicators)
ORDER BY percapita DESC
"""

resultset1 = %sql $query
resultdf1 = resultset1.DataFrame()
resultdf1.head()

 * sqlite:///../../dbfiles/book.db
Done.


,code,year,percapita
0,MCO,2008,216.000000
1,MCO,2007,195.666667
2,MCO,2014,176.500000
3,LIE,2014,166.500000
4,MCO,2013,163.750000


In [10]:
# Testing cell
assert len(resultdf1) == 12862

**Q2:** Using the table `indicators`, select all columns, plus a new column named `new` that tells whether `exports` are larger than `imports`.  Sort in descending order of gdp.

In [19]:
query = """
SELECT *, exports > imports AS new FROM indicators
ORDER BY gdp DESC
"""


resultset2 = %sql $query
resultdf2 = resultset2.DataFrame()
resultdf2.head()

 * sqlite:///../../dbfiles/book.db
Done.


,year,code,pop,gdp,life,cell,imports,exports,new
0,2018,USA,327.17,20494.1,NaN,NaN,NaN,NaN,NaN
1,2017,USA,325.15,19485.4,78.5,391.60,2342670.0,1545610.0,0.0
2,2016,USA,323.07,18707.2,78.5,387.40,2189180.0,1453700.0,0.0
3,2015,USA,320.74,18219.3,78.7,382.31,2241660.0,1504570.0,0.0
4,2014,USA,318.39,17521.8,78.8,355.50,2347680.0,1620480.0,0.0


In [20]:
# Testing cell
assert resultdf2.shape == (12862,9)
assert resultdf2.loc[2,'new'] == 0.0

### Switching to the `school` Database

The following cell, **assuming `dbsource` is defined from above**, creates a new connection string and establishes a new connection to the `school` database.

In [21]:
scheme, dbdir, database = getsqlite_creds(source="sqlite2")
template = '{}:///{}/{}.db'
cstring = template.format(scheme, dbdir, database)
print("Connection string:", cstring)

%sql $cstring

Connection string: sqlite:///../../dbfiles/school.db


**Q3:** Using the `instructors` table in the `school` database, compute and project a single new column `full_name` obtained as the last name, followed by a comma and a space, followed by the first name.  Put the names in alphabetical order.

In [31]:
query = """
SELECT instructorlast || ',' || ' ' || instructorfirst AS full_name
FROM instructors
ORDER BY full_name ASC
"""


resultset3 = %sql $query
resultdf3 = resultset3.DataFrame()
resultdf3.head()

   sqlite:///../../dbfiles/book.db
 * sqlite:///../../dbfiles/school.db
Done.


,full_name
0,"Aguilar, Stephen"
1,"Anderson, Philip"
2,"Arnaud, Antoine"
3,"Austin, Stephanie"
4,"Bailey, Jayden"


In [32]:
# Testing cell
assert len(resultdf3) == 292
assert 'Aguilar, Stephen' in list(resultdf3['full_name'])

**Q4:** Using the `courses` table in the `school` database, select a new column `catalog` obtained as the course subject, followed by a space, followed by the course number, followed by a colon and a space, followed by the course title. Only include distinct catalogue entries and don't include any NULL entries.

In [37]:
query = """
SELECT coursesubject || ' ' || coursenum || ':' || ' ' || coursetitle AS catalog
FROM courses
WHERE catalog IS NOT NULL
"""

resultset4 = %sql $query
resultdf4 = resultset4.DataFrame()
resultdf4.head()

   sqlite:///../../dbfiles/book.db
 * sqlite:///../../dbfiles/school.db
Done.


,catalog
0,ARAB 111: Beginning Arabic I
1,ARAB 112: Beginning Arabic II
2,ARAB 211: Intermediate Arabic I
3,ARAB 361: Directed Study
4,ARTH 151: Intro to Art History


In [38]:
# Testing cell
assert len(resultdf4) == 904
assert 'WMST 101: Issues in Feminism' in list(resultdf4['catalog'])